In [70]:
import pandas as pd
from sodapy import Socrata
from datetime import datetime
import pickle
from geopy import distance
import numpy as np
import time

In [71]:
start = time.time()

dict_dfs={"grad":["nb39-jx2v"], "trees":["uvpi-gqnh"],"crashes":["h9gi-nx95"],"pc":["cuzb-dmcd"],
          "shootings":["833y-fsy8"], "arrests":["8h9b-rp9u"],"programs":["mbd7-jfnc"],"bins":["sxx4-xhzg"]}
for key in dict_dfs:
    x=dict_dfs[key][0]
    dict_dfs[key]={"key":x,"last_updated":0,"data":""}
    
dict_dfs

{'grad': {'key': 'nb39-jx2v', 'last_updated': 0, 'data': ''},
 'trees': {'key': 'uvpi-gqnh', 'last_updated': 0, 'data': ''},
 'crashes': {'key': 'h9gi-nx95', 'last_updated': 0, 'data': ''},
 'pc': {'key': 'cuzb-dmcd', 'last_updated': 0, 'data': ''},
 'shootings': {'key': '833y-fsy8', 'last_updated': 0, 'data': ''},
 'arrests': {'key': '8h9b-rp9u', 'last_updated': 0, 'data': ''},
 'programs': {'key': 'mbd7-jfnc', 'last_updated': 0, 'data': ''},
 'bins': {'key': 'sxx4-xhzg', 'last_updated': 0, 'data': ''}}

In [72]:
def get_data(data_dict):
    data_url = "data.cityofnewyork.us"
    token = "HnDwBjZUCabVQojpbl6aIDrDI"
    client = Socrata(data_url,token, timeout=120)
    for key in data_dict:
        data_set=data_dict[key]["key"]
        meta = client.get_metadata(data_set)
        if meta["viewLastModified"]>data_dict[key]["last_updated"]:
            data_dict[key]["last_updated"]=meta["viewLastModified"]
            data_dict[key]["data"]=client.get(data_set, limit=10000)
            lastUpdated=True
        print(key + " done")      
    
     

get_data(dict_dfs)


grad done
trees done
crashes done
pc done
shootings done
arrests done
programs done
bins done


In [73]:
def get_dataframes(data_dict):
    for key in data_dict:
        if key in ["addresses","scatter"]:
            continue
        df=pd.DataFrame(data_dict[key]["data"])
        columns=df.columns
        if "latitude" in df.columns:
            df["location"]=list(zip(df["latitude"].to_list(),df["longitude"].to_list()))
            df=df[["location"]]
            df=df[df.location.notna()]
        if "lat" in df.columns:
            df["location"]=list(zip(df["lat"].to_list(),df["lon"].to_list()))
            df=df[["location"]]
            df=df[df.location.notna()]
        data_dict[key]["df"]=df
        print(key + " done")
    df=pd.read_csv("data/2016_DOE_High_School_Directory-2.csv")
    df["location"]=list(zip(df["Latitude"].to_list(),df["Longitude"].to_list()))
    df=df[["dbn","school_name","borough","location"]]
    data_dict["grad"]["df"]=data_dict["grad"]["df"].merge(df,on="dbn",how="left")
    data_dict["scatter"]={"df":df}
get_dataframes(dict_dfs)

grad done
trees done
crashes done
pc done
shootings done
arrests done
programs done
bins done


In [81]:
def prep_grad(data_dict):
    df_school=data_dict["grad"]["df"]
    df_school=df_school[df_school.demographic_category=="All Students"]
    data_dict["grad"]["df"]=df_school
    
prep_grad(dict_dfs)


In [74]:
def get_distance(loc1,loc2):
    try:
        return distance.distance(loc1,loc2).meters
    except:
        return np.nan
    

def get_numFeat(dict_dfs):
    dbns=dict_dfs["grad"]["df"]["dbn"].to_list()
    df_school=dict_dfs["grad"]["df"]
    df_school=df_school[df_school.location.notnull()]
    dbns=set(df_school["dbn"].to_list())
    dict_dbn={}
    dict_keys={}
    dict_feat={}
    dict_dfs["scatter"]["feat"]=""
    
    for key in dict_dfs:
        if key != "scatter":
            dict_dbn={}
            print( "\n\n\n",key)
            if key != "grad":
                df=dict_dfs[key]["df"]
                df=df[df.location.notnull()]
                for dbn in dbns:
                    locSchool=df_school.loc[df_school["dbn"]==dbn,"location"].to_list()[0]
                    df["distance"]=df["location"].apply(lambda x: get_distance(locSchool,x))
                    numFeat=(df["distance"].values<500).sum()
                    dict_dbn[dbn]=numFeat
                dict_keys[key]=dict_dbn
    #dict_feat["feat"]=dict_keys
    dict_dfs["scatter"]["feat"]=dict_keys
                   
                    
            
 
    



get_numFeat(dict_dfs)

end=time.time()

print("/n/n",(end-start)/59.992)




 grad



 trees



 crashes



 pc



 shootings



 arrests



 programs



 bins
/n/n 29.736872819570372


In [82]:
def get_scatterDf(data_dict):
    dfScatter=data_dict["scatter"]["df"]
    dfSchool=data_dict["grad"]["df"][["dbn", "demographic_category", "cohort_year", "total_grads_of_cohort"]]
    
    dfFeat=pd.DataFrame.from_dict(data_dict["scatter"]["feat"])
    dfFeat.reset_index(inplace=True)
    dfFeat=dfFeat.rename(columns={"index":"dbn"})
    dfScatter=dfScatter.merge(dfFeat,on="dbn",how="left")
    dfScatter=dfScatter.merge(dfSchool,on="dbn",how="left")
    return dfScatter



x=get_scatterDf(dict_dfs)

x.to_csv("data/dataForScatter.csv")

In [83]:
x.isnull().sum()

dbn                        0
school_name                0
borough                    0
location                   0
trees                    237
crashes                  237
pc                       237
shootings                237
arrests                  237
programs                 237
bins                     237
demographic_category     239
cohort_year              239
total_grads_of_cohort    673
dtype: int64